In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 26.3 MB/s 
     |████████████████████████████████| 596 kB 45.6 MB/s 
     |████████████████████████████████| 101 kB 3.3 MB/s 
     |████████████████████████████████| 6.6 MB 50.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
!pip install hazm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 316 kB 23.5 MB/s 
     |████████████████████████████████| 1.4 MB 46.6 MB/s 
     |████████████████████████████████| 233 kB 47.7 MB/s 
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394488 sha256=b261e8584895be36ec92946bf1c00b46f98ea5e5097bfb6de04de9b74051cddf
  Stored in directory: /root/.cache/pip/wheels/9b/fd/0c/d92302c876e5de87ebd7fc0979d82edb93e2d8d768bf71fac4
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=154705 sha256=648aba6a8a03415acca6f258cab0f4e24cc064a81eba210c1d88cc4a48210021
  Stored in directory: /root/.cache/pip/wheels/ab/b2/5b/0fe4b8f5c0e65341e8ea7bb3f4a6ebabfe8b1ac31322392dbf
Successfully built nltk libwapiti
  Attempting uninstall: nltk
    Found existing installation: nltk 3.7
    Uninstalling nltk-3.7:
      Successfully uninstalled nltk-3.7


In [ ]:
import torch
import math
import numpy as np
import os
from hazm import *

In [6]:
from transformers import pipeline, AutoTokenizer, GPT2LMHeadModel
tokenizer = AutoTokenizer.from_pretrained('bolbolzaban/gpt2-persian')
model = GPT2LMHeadModel.from_pretrained('bolbolzaban/gpt2-persian')
generator = pipeline('text-generation', model, tokenizer=tokenizer, config={'max_length':256})
sample = generator('در یک اتفاق شگفت انگیز، پژوهشگران')

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.30k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/524k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Downloading:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:9 for open-end generation.


In [7]:
def score(tokens_tensor):
    loss=model(tokens_tensor, labels=tokens_tensor)[0]
    return np.exp(loss.detach().numpy())

In [8]:
# calculate propbablity of a sentence using GPT2
def calc_score(text):
  tokens_tensor = tokenizer.encode(text, add_special_tokens=False, return_tensors="pt")           
  return score(tokens_tensor)

In [9]:
from hazm import *
normalizer = Normalizer()

In [10]:
#normalize using Hazm
def normalize(texts):
  for i in range(len(texts)):
    texts[i] = normalizer.normalize(texts[i])
  return texts

In [11]:
texts = ['را کرد شدیدی امروز بیت‌کوین سقوط تجربه', 'امروز بیت‌کوین سقوط شدیدی را تجربه کرد']
print(normalize(texts))

['را کرد شدیدی امروز بیت\u200cکوین سقوط تجربه', 'امروز بیت\u200cکوین سقوط شدیدی را تجربه کرد']


In [12]:
import time
#test sentence probablities -> the lower number the more probable
start = time.time()
for txt in texts:
  print(txt, calc_score(txt))
end = time.time()
print('elapsed: ',end - start)

را کرد شدیدی امروز بیت‌کوین سقوط تجربه 5168.14
امروز بیت‌کوین سقوط شدیدی را تجربه کرد 70.475235
elapsed:  0.9591860771179199


In [13]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
%cd /content/drive/MyDrive

/content/drive/MyDrive


In [15]:
os.getcwd()

'/content/drive/MyDrive'

In [16]:
tagger = POSTagger(model='resources/postagger.model')

In [17]:
chunker = Chunker( model= 'resources/chunker.model' )


In [18]:
from itertools import permutations
def get_permutations(sentence):
   """
   find all combinations of a sentence
   """
   words = list(sentence.split())
   permute = permutations(words)
   output = []
   for i in permute:
        permutelist = list(i)
        string = ""
        for j in permutelist:
          string += (j + " ")
        output.append(string)
   return output


In [19]:
def filter_combinations(combs):
  out = []
  for comb in combs:
    tagged = tagger.tag(word_tokenize(comb))
    #parsed = tree2brackets( chunker.parse (tagged))
    if any(t[1] == 'V' for t in tagged):
      if tagged[-1][1] == 'V' and (tagged[0][1] == 'N' or tagged[0][1] == 'ADV'):
        #print('--->',tagged, '---->', parsed)
        out.append(comb)
  return out

In [20]:
tagged = tagger.tag(word_tokenize('امروز بیت‌کوین سقوط شدیدی را تجربه کرد'))
tagged[-1][1]

'V'

In [21]:
### remove all sentences that don't finish with verb
lst = get_permutations('امروز بیت‌کوین سقوط شدیدی را تجربه کرد')
print(len(lst))
lst = filter_combinations(lst)
print(len(lst))

5040
204


In [22]:
THRESHOLD = 60
CUTOFF_COUNT = 2
def get_scores_list(sentence):
  """
  calculate probablity scores for all combinations of a sentence
  returns: sorted list of combinations with probablities
  """
  
  combs = get_permutations(sentence)
  #print(len(combs))
  combs = filter_combinations(combs)

  count = 0  ## keeps the number of scores less than 100

  sent_score = []
  for comb in combs:
    score = calc_score(comb)
    sent_score.append((comb, score))  
    if score < 100:
      count += 1
    if score < THRESHOLD or score == CUTOFF_COUNT:
      break

  sorted_sent_score = sorted(sent_score, key= lambda a: a[1])
  return sorted_sent_score

In [23]:
get_scores_list('دل از رفت دست')

[('دل از دست رفت ', 35.436897)]

In [24]:
def get_most_likely(n, sentence):
  """
  finds n most likely combinations for a sentence 
  """
  return get_scores_list(sentence)[0:n]

In [25]:
def print_sentence_score(lst):
  for i in l: print(i[0], i[1])
  

In [26]:
l = get_most_likely(2, 'را کرد شدیدی امروز بیت‌کوین سقوط تجربه')

In [27]:
l[0][0]

'امروز بیت\u200cکوین سقوط شدیدی را تجربه کرد '

In [28]:
print_sentence_score(l)

امروز بیت‌کوین سقوط شدیدی را تجربه کرد  70.475235
امروز سقوط شدیدی بیت‌کوین را تجربه کرد  121.6243


In [36]:
test_list = []
with open('data/test.txt', 'r', encoding='utf8') as file:
  for line in file:
    test_list.append(sent_tokenize(str(line)))

In [37]:
len(test_list)

20

In [38]:
import random
def scramble(sent):
    words = sent.split()
    random.shuffle(words)
    return ' '.join(words)

In [32]:
scramble(test_list[0][0])

'این کار سخت است بسیار'

In [39]:
test_set = []
for elem in test_list:
    s = scramble(elem[0])
    test_set.append( (s, elem[0]) )

In [40]:
test_set

[('کار سخت بسیار این است', 'این کار بسیار سخت است'),
 ('بهانه تمام است این ها حرف', 'تمام این حرف ها بهانه است'),
 ('پیدا یک فرد کن مناسب', 'یک فرد مناسب پیدا کن'),
 ('است ایران اینجا', 'اینجا ایران است'),
 ('سرزمین پاک مردان', 'سرزمین مردان پاک'),
 ('است سرپیچی بزرگترین اشتباه', 'بزرگترین اشتباه سرپیچی است'),
 ('برای آمده من ام کار اینجا', 'من برای کار اینجا آمده ام'),
 ('بازی دیدم دیشب را', 'بازی دیشب را دیدم'),
 ('چنین یک تواند کند می قهرمان', 'یک قهرمان می تواند چنین کند'),
 ('است یک پدیده\u200cی بیتکوین بزرگ', 'بیتکوین یک پدیده\u200cی بزرگ است'),
 ('ارز به دیجیتال خواهد\u200c پا کرد انقلابی',
  'ارز دیجیتال انقلابی به پا خواهد\u200c کرد'),
 ('اقتصادی دارد رشد ایران رتبه در', 'ایران در رشد اقتصادی رتبه دارد'),
 ('وضعیت کشور مطلوب نیست سیاسی', 'وضعیت سیاسی کشور مطلوب نیست'),
 ('امروزه دارد مناسبی جایگاه دانشگاه', 'امروزه دانشگاه جایگاه مناسبی دارد'),
 ('ایران اینجا است', 'اینجا ایران است'),
 ('هوای خوب نیست شهر', 'هوای شهر خوب نیست'),
 ('مرد قوی تواند شود می برنده', 'مرد قوی می توان

In [41]:
preds = []
for elem in test_set:
  print(elem[0])
  lst = get_most_likely(2, elem[0])
  preds.append(lst)

کار سخت بسیار این است
بهانه تمام است این ها حرف
پیدا یک فرد کن مناسب
است ایران اینجا
سرزمین پاک مردان
است سرپیچی بزرگترین اشتباه
برای آمده من ام کار اینجا
بازی دیدم دیشب را
چنین یک تواند کند می قهرمان
است یک پدیده‌ی بیتکوین بزرگ
ارز به دیجیتال خواهد‌ پا کرد انقلابی
اقتصادی دارد رشد ایران رتبه در
وضعیت کشور مطلوب نیست سیاسی
امروزه دارد مناسبی جایگاه دانشگاه
ایران اینجا است
هوای خوب نیست شهر
مرد قوی تواند شود می برنده
وطن جاییست که احساس کنی آرامش
باید پخت غذا برای داغ باشد گاز
جا پا ای وطنم بر شکوه


In [42]:
preds

[[('بسیار سخت این کار است ', 89.29694), ('بسیار این کار سخت است ', 96.73555)],
 [('ها این تمام حرف بهانه است ', 686.80585),
  ('ها تمام این حرف بهانه است ', 759.56964)],
 [('فرد یک پیدا مناسب کن ', 7560.378)],
 [('اینجا ایران است ', 123.96689), ('ایران اینجا است ', 1131.995)],
 [],
 [('سرپیچی بزرگترین اشتباه است ', 713.2584),
  ('اشتباه بزرگترین سرپیچی است ', 3442.4644)],
 [('ام من برای کار اینجا آمده ', 209.6203),
  ('ام برای کار من اینجا آمده ', 212.26343)],
 [('دیشب بازی را دیدم ', 60.491295), ('بازی را دیشب دیدم ', 187.37434)],
 [('قهرمان یک کند چنین می تواند ', 493.60052),
  ('قهرمان چنین یک کند می تواند ', 530.3212)],
 [('بیتکوین یک پدیده\u200cی بزرگ است ', 107.00952),
  ('بیتکوین یک بزرگ پدیده\u200cی است ', 720.24286)],
 [('انقلابی به ارز دیجیتال خواهد\u200c پا کرد ', 950.74225),
  ('ارز دیجیتال کرد انقلابی به پا خواهد\u200c ', 974.9533)],
 [('رتبه در رشد اقتصادی ایران دارد ', 77.90701),
  ('رتبه در ایران رشد اقتصادی دارد ', 154.07645)],
 [('کشور وضعیت سیاسی مطلوب نیست ', 314.14